In [1]:
import sisl
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Quantum Hall Effect
-----------------------------

In this exercise, we will build on [TB_07](../TB_07/run.ipynb) to simulate the quantum hall effect.

Here, we will extract the Hall resistance from the transmissions calculated with TBtrans using the Landauer-Büttiker formalism.

## Exercise Overview:
1. Create a Hall bar
2. Construct Hamiltonians and add magnetic fields (see [TB_07](../TB_07/run.ipynb))
3. Calculate the transmission with `tbtrans`
4. Extract the Hall resistance $R_H$.

## Exercise

### 1. Create a Hall bar
In order to be able to observe the quantum hall effect, the size of the hall bar needs to be big enough. For a 4(6) lead device reasonable dimensions are: 
1. 4-lead device:
   - Width of electrodes (perpendicular to the semi-infinite axis): 30 atoms
   - Offset of the electrodes 2(3) from the corner of the central part: >= 5 atoms


2. 6-lead device: 
   - Width of electrodes (perpendicular to the semi-infinite axis): 30 atoms
   - Spacing between electrodes on the same side: 50 atoms
   - Offset of the electrodes 2(3,5,6) from the corner of the central part: > 15 atoms

|4-lead device |6-lead device |
|:-------------|:-------------|
|<img src="img/set-up-4.png" alt="4 lead hall bar" style="width: 310px;"/>|<img src="img/set-up-6.png" alt="6 lead hall bar" style="width: 490px;"/>|

In [ ]:
# Create a Hall bar

### 2. Construct Hamiltonian and add magnetic fields

The required field strengths may vary depending on the size of the hall bar.
A good starting point might by `B = 1 / np.arange(1,31)`

In [ ]:
def peierls(B=0):
    def peierls(self, ia, atoms, atoms_xyz=None):
        idx = self.geometry.close(ia, R=[0.1, 1.01], atoms=atoms, atoms_xyz=atoms_xyz)
        # Onsite
        self[ia, idx[0]] = 4
    
        # Hopping
        if B == 0:
            self[ia, idx[1]] = -1
        else:
            xyz = self.geometry.xyz[ia]
            dxyz = self.geometry[idx[1]]
            self[ia, idx[1]] = - np.exp(-0.5j * B * (dxyz[:, 0] - xyz[0])*(dxyz[:,1] + xyz[1]))
    return peierls

In [ ]:
# H0 = sisl.Hamiltonian(geom, dtype=np.float64)
# H0.construct(peierls()))
#
# HB = sisl.Hamiltonian(geom, dtype=np.complex128)
# HB.construct(peierls(B))
# 
# dH = ... 

### 3. Calculate the transmission with TBtrans

The folder of this exercise contains the skeleton of an input file for a 4-lead (`RUN-4.fdf`) and 6-lead device (`RUN-6.fdf`), as well as a script to run TBtrans for all values of the magnetic field (`run.sh`).

*Depending on the size of the Hall bar this step might require some a considerable amount of time.*

### 4. Extract the Hall resistance $(R_H)$

The Hall resistance ($R_H$) in a 4 lead Hall bar like the one shown above is given by

$$R_H = \left.\frac{V_2-V_3}{I_1}\right|_{I_2 = I_3 = 0}.$$

In order to find a relationship between the transmission curves and the hall resistance, we express the lead currents $I_i$ in terms of applied biases $V_i$ and the transmissions $T_{ij}$ between leads $i$ and $j$
$$I_i = \sum_j G_{ij} (V_i - V_j)\quad\text{where}\quad G_{ij} = \frac{2e^2}{h} T_{ij}.$$
Since the currents only depend on bias differences, we can set one of them to zero without loss of generality (here $V_3 = 0$). This allows us to rewrite the relation as

$$\mathbf{I} = \mathcal{G} \mathbf{V}
\quad\text{, where}\quad
\mathcal{G}_{ii} = \sum_{i\neq j} G_{ij}
\quad\text{and}\quad
\mathcal{G}_{ij} = - G_{ij}.$$

Using the inverse $\mathbf{R}$ of $\mathcal{G}$, we can express $V_2$ and $V_3$ in terms of the lead currents $I_i$ and calculate the Hall conductance:

$$
V_i = R_{i1} I_1 + R_{i2} I_2 + R_{i3} I_3,
$$
and finally, we find the Hall resistance:
$$
R_H = R_{21}-R_{31}
$$

The derivation for the 6-lead device is analogous and yields:
$$
R_H = R_{21}-R_{61}
$$

In [ ]:
# Create short-hand function to open files
gs = sisl.get_sile
# No magnetic field
tbt0 = gs('M_0/siesta.TBT.nc')
# All magnetic fields in increasing order
tbts = [gs('M_{}/siesta.TBT.nc'.format(rec_phi)) for rec_phi in rec_phis]

In [ ]:
def G_matrix(tbtsile):
    # Construct G
  
G0 = G_matrix(tbt0)
# Remove one row and column from the matrix
G0 = np.delete(np.delete(G0, ..., axis=...), ..., axis=...)
R0 = np.linalg.inv(G)

G = ...
R = ...

RH = ...

In [ ]:
# Plot the hall resitance
#  - as a function of energy for a fixed magnetic field strength
#  - as a funciton of the magnetic field strength for a fixed energy